In [ ]:
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [ ]:
!pip install pretty_midi
!pip install --upgrade pyfluidsynth

In [ ]:
base_dir = "/kaggle/input/classical-music-midi"
all_midis= []

# for artist_dir in os.listdir(base_dir):
#     for directory, _, files in os.walk(base_dir+"/"+artist_dir):
#         for file in files:
#             full_path = f"{directory}/{file}"
#             midi = converter.parse(full_path)
#             all_midis.append(midi)

base_dir = "/kaggle/input/classical-music-midi/mozart"

for directory, _, files in os.walk(base_dir):
    for file in files:
        full_path = f"{directory}/{file}"
        midi = converter.parse(full_path)
        all_midis.append(midi)

In [ ]:
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= extract_notes(all_midis)

In [ ]:
import pretty_midi
import IPython.display as ipd

def play_audio(path):
    pm = pretty_midi.PrettyMIDI(path)
    audio_signal = pm.fluidsynth()
    ipd.Audio(audio_signal, rate=pm.get_end_time())
    

play_audio("/kaggle/input/classical-music-midi/mozart/mz_332_3.mid")

In [ ]:
Corpus[:20]

In [ ]:
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

Melody_Snippet = chords_n_notes(Corpus[:100])